In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

In [28]:
df = pd.read_csv(r"D:\D-drive\Coding\Major-Project\new_\data\preprocessed_data.csv")

In [29]:
X = df[["I","P","Q","T","Hydrogen","Oxygen","RH anode","Rh Cathode"]].values
y = df["V"].values

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
def create_ann():
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='linear'))  # regression output
    model.compile(optimizer='adam', loss='mse')
    return model

ann_regressor = KerasRegressor(
    build_fn=create_ann,
    epochs=100,
    batch_size=16,
    verbose=0
)

In [33]:
svr = SVR(kernel="rbf", C=10, epsilon=0.1)

In [34]:
stack_model = StackingRegressor(
    estimators=[
        ("ann", ann_regressor),
        ("svr", svr)
    ],
    final_estimator=LinearRegression(),
)

In [35]:
stack_model.fit(X_train_scaled, y_train)
y_pred = stack_model.predict(X_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Stacked ANN + SVR RMSE:", rmse)

ValueError: The estimator KerasRegressor should be a regressor.